In [9]:
"""
This training script can be run both on a single gpu in debug mode,
and also in a larger training run with distributed data parallel (ddp).

To run on a single GPU, example:
$ python train.py --batch_size=32 --compile=False

To run with DDP on 4 gpus on 1 node, example:
$ torchrun --standalone --nproc_per_node=4 train.py

To run with DDP on 4 gpus across 2 nodes, example:
- Run on the first (master) node with example IP 123.456.123.456:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=0 --master_addr=123.456.123.456 --master_port=1234 train.py
- Run on the worker node:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=1 --master_addr=123.456.123.456 --master_port=1234 train.py
(If your cluster does not have Infiniband interconnect prepend NCCL_IB_DISABLE=1)
"""

import os
import time
import math
import pickle
import json
from contextlib import nullcontext

import pandas as pd

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out-stock'
eval_interval = 100
log_interval = 1
eval_iters = 20
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = False # if True, always save a checkpoint after each eval
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False # disabled by default
wandb_project = 'stock'
wandb_run_name = 'mini-gpt' # 'run' + str(time.time())
# data
dataset = 'stock'
gradient_accumulation_steps = 1 # used to simulate larger batch sizes
batch_size = 12 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 16
# model
n_layer = 1
n_head = 2
n_embd = 12
dropout = 0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?
meta_vocab_size = n_embd // 2

# adamw optimizer
learning_rate = 1e-3 # max learning rate
max_iters = 30000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.99
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 100 # how many steps to warm up for
lr_decay_iters = 2000 # should be ~= max_iters per Chinchilla
min_lr = 1e-4 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla

# system
device = 'cpu' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

master_process = True
seed_offset = 0


if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)


In [10]:
np.random.seed(42)

datadir = os.path.join('data', dataset)

# meta数据
meta = {}
with open(os.path.join(datadir, 'meta.pkl'), 'r') as f:
    meta = json.load(f)
def decode(id):
    return meta['itos'][str(id)]


pd_train_data = pd.read_csv(os.path.join(datadir, 'train.csv')).iloc[-1000:,:]
pd_val_data = pd.read_csv(os.path.join(datadir, 'val.csv'))

# 将NumPy数组转换为pandas DataFrame
train_data = torch.tensor(pd_train_data.iloc[:, 1:n_embd+1].values, dtype=torch.float16)
val_data = torch.tensor(pd_val_data.iloc[:100, 1:n_embd+1].values, dtype=torch.float16)
print(train_data.shape)
torch.manual_seed(33)

def print_data_info(split, index_data, index_id):
    data = pd_train_data if split == 'train' else pd_val_data
    print(f'{split}, code is {decode(index_id)}, date is {data.iloc[index_data, 0]},price_chg is {data.iloc[index_data, (index_id+1)*2]}, aount_chg is {data.iloc[index_data, (index_id+1)*2-1]}')


def get_batch(split):
    data = train_data if split == 'train' else val_data

    indices = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in indices])
    y_indices = [torch.arange(0, len(x[i][0]) // 2) for i, j in enumerate(indices)]
    
    y_list = []
    for i, j in enumerate(indices):
        next_row = data[j + 1:j + 1 + block_size]
        price_chg_cols = [2 * col + 1 for col in y_indices[i]]
        
        topk = 1
        topk_values, topk_indices = next_row[:, price_chg_cols].topk(topk, dim=1)

        random_indices = torch.randint(0, topk, (topk_indices.size(0),))
        selected_indices = topk_indices[torch.arange(topk_indices.size(0)), random_indices]

        y_list.append(selected_indices.clone().detach())
        if i ==0 :
            print_data_info(split, (j+1).item(), selected_indices[0].item())
        
        

    y = torch.stack(y_list)

    x, y = x.to(device), y.to(device)
    return x, y

X, Y = get_batch('train')
print(f'X is {X[0,0]}, Y is {Y[0,0]}, Y code is {decode(Y[0,0].item())}, Y.value is {X[0,1,(Y[0,0])*2+1]}, X.next is {X[0,1]}')
# _,_ = get_batch('train')




torch.Size([1000, 12])
train, code is 000001.SZ, date is 20200817,price_chg is 1.0498, aount_chg is 3.1191
X is tensor([1.3262, 1.0205, 1.1465, 1.0039, 1.0674, 0.9951, 1.0195, 1.0068, 0.9233,
        1.0098, 1.1543, 1.0195], dtype=torch.float16), Y is 0, Y code is 000001.SZ, Y.value is 1.0498046875, X.next is tensor([3.1191, 1.0498, 1.4688, 1.0195, 1.5000, 1.0176, 3.9121, 1.0342, 1.5430,
        1.0254, 0.6777, 0.9990], dtype=torch.float16)


In [11]:
# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=meta_vocab_size, dropout=dropout) # start with model_args from command line
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)


model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)

checkpoint = None # free up memory



# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log and master_process:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

X, Y = get_batch('train')

# 遍历并打印模型中的所有权重参数
# for name, param in model.named_parameters():
#     print(f"{name}: {param.shape}, {param}")

print(f'X is {X}')
print(f'Y is {Y}')
logits, loss = model(X, Y)
print(logits)
# print(f'logits is {logits}, shape is {logits.shape}')
# print(f'loss is {loss}, loss.shape is {loss.shape}')


# print(model)


Initializing a new model from scratch
number of parameters: 0.00M
num decayed parameter tensors: 6, with 1,992 parameters
num non-decayed parameter tensors: 3, with 36 parameters
using fused AdamW: False
train, code is 000006.SZ, date is 20200610,price_chg is 1.0217, aount_chg is 1.1455
X is tensor([[[0.8066, 1.0039, 1.0254,  ..., 0.9771, 0.3884, 0.9976],
         [1.2090, 0.9868, 1.1191,  ..., 1.0215, 0.7725, 0.9990],
         [2.2695, 0.9697, 0.7808,  ..., 0.9883, 0.9531, 0.9976],
         ...,
         [0.9014, 1.0000, 0.8057,  ..., 1.0332, 0.7964, 0.9907],
         [1.8369, 1.0254, 3.8262,  ..., 1.0996, 1.1250, 1.0059],
         [1.5586, 1.0234, 0.8467,  ..., 0.9985, 0.6704, 0.9907]],

        [[0.9775, 0.9922, 1.7500,  ..., 1.0234, 0.4399, 0.9927],
         [1.2881, 1.0020, 0.9712,  ..., 1.0098, 1.0469, 0.9917],
         [0.7866, 0.9800, 0.5249,  ..., 0.9863, 0.6812, 0.9971],
         ...,
         [0.6958, 0.9922, 0.8408,  ..., 1.0068, 1.0244, 1.0088],
         [0.8784, 1.0078, 0

/Users/renyabin/opt/anaconda3/envs/nanogpt/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [12]:

# training loop
X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model # unwrap DDP container if needed
running_mfu = -1.0
while True:

    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": losses['train'],
                "val/loss": losses['val'],
                "lr": lr,
                "mfu": running_mfu*100, # convert to percentage
            })
        print('ca')
        if losses['val'] < best_val_loss or always_save_checkpoint:
            print('save啊')
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
    if iter_num == 0 and eval_only:
        break

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):
        with ctx:
            # print(f'X is {X[0,0]}, Y is {Y[0,0]}, Y.value is {X[0,1,Y[0,0]]}, X.next is {X[0,1]}')
            # print(f'----{iter_num}')
            # print(f'X is {X}')
            # print(f'Y is {Y}')
            logits, loss = model(X, Y)
            # print(f'loss is {loss}')
            # print(f'logits is {logits[0]}')
            
            loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        X, Y = get_batch('train')
        # X, Y = X, Y
        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward()
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms")
    iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break




train, code is 000007.SZ, date is 20201103,price_chg is 1.0299, aount_chg is 1.3472
train, code is 000004.SZ, date is 20190507,price_chg is 1.0778, aount_chg is 0.9917
train, code is 000002.SZ, date is 20190912,price_chg is 1.0323, aount_chg is 2.6232
train, code is 000004.SZ, date is 20210727,price_chg is 1.017, aount_chg is 0.9827
train, code is 000007.SZ, date is 20190716,price_chg is 1.0201, aount_chg is 0.9596
train, code is 000004.SZ, date is 20200206,price_chg is 1.0303, aount_chg is 0.8774
train, code is 000002.SZ, date is 20181101,price_chg is 1.0078, aount_chg is 1.1272
train, code is 000006.SZ, date is 20211123,price_chg is 1.0097, aount_chg is 0.9124
train, code is 000002.SZ, date is 20190717,price_chg is 1.0023, aount_chg is 0.6703
train, code is 000001.SZ, date is 20191029,price_chg is 1.015, aount_chg is 0.7424
train, code is 000002.SZ, date is 20200214,price_chg is 1.0284, aount_chg is 1.3493
train, code is 000001.SZ, date is 20200508,price_chg is 1.019, aount_chg is 1.

KeyboardInterrupt: 